

> CONECTANDO COM O GOOGLE



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Função para tratamentos

In [0]:
def removeCaracteres(nomeColuna, dataSet):
  dataSet[nomeColuna]= dataSet[nomeColuna].astype(str)
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace('[', '')
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace(']', '')
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace('"', '')
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace('.', '')
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace('_', '')
  dataSet[nomeColuna] = dataSet[nomeColuna].str.lower()


Carrega base e aplica tratamentos

In [0]:
import os
workdir_path = '/content/drive/My Drive/' + '00 PUC BI MASTER/00 - PROJ (TCC)/PREDIÇÃO ATIVIDADES/event2mind'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

In [0]:
import pandas as pd
import numpy as np
event2mind = pd.read_csv('total.csv', sep = ';')

removeCaracteres('Xintent', event2mind)
removeCaracteres('Xemotion', event2mind)

#Criando chave (Indice)
event2mind['Indice'] = list(range(len(event2mind.index)))
event2mind['Indice'] = event2mind['Indice'] + 1

Gerando novas linhas para as intenções separadas por vírgulas

In [0]:
event2mindTransposed = pd.DataFrame(event2mind.Xintent.str.split(',').tolist(), index = event2mind.Indice).stack()
event2mindTransposed = event2mindTransposed.reset_index([0,'Indice'])
event2mindTransposed.columns = ['Indice','Xintent']

In [0]:
event2mindTransposed.head()

,Indice,Xintent
0,1,none
1,2,none
2,3,none
3,4,none
4,5,none


Verificando sem intenções não preenchidas

In [6]:
cleanDf = event2mindTransposed[event2mindTransposed['Xintent'] != 'none']

cleanDf.head(10)

,Indice,Xintent
772,773,to have an advantage
773,773,to fulfill a desire
774,773,to get out of trouble
775,774,to be powerful
776,775,to be influential
777,776,to reform men with a bad attitude
778,777,good about themselves
779,778,to be respectful
780,779,to show gratitude
781,779,to be polite


Quebrando as emoções separadas por vírgula nas linhas

In [0]:
event2mindTransposed2 = pd.DataFrame(event2mind.Xemotion.str.split(',').tolist(), index = event2mind.Indice).stack()
event2mindTransposed2 = event2mindTransposed2.reset_index([0,'Indice'])
event2mindTransposed2.columns = ['Indice','Xemotion']

In [8]:
cleanDf = event2mindTransposed2[event2mindTransposed2['Xemotion'] != 'none']

cleanDf.head(10)

,Indice,Xemotion
772,773,pleased
773,773,smug
774,773,excited
775,774,powerful
776,775,powerful
777,776,satisfied about helping mankind
778,777,good
779,778,grateful
780,779,satisfied
781,780,vengeance


Unificando a base com duplicadas

In [0]:
event2mindAux = event2mindTransposed.join(event2mindTransposed2.set_index('Indice'), on = 'Indice', how = 'left')

event2mindFinal = event2mindAux.join(event2mind[["Indice", "Event", "Xsent", "Osent"]].set_index('Indice'), on = 'Indice', how = 'left')

event2mindFinal['Indice2'] = list(range(len(event2mindFinal)))
event2mindFinal['Indice2'] = event2mindFinal['Indice2'] + 1
event2mindFinal.set_index('Indice2', inplace=True)

event2mindFinal['Event_2'] = event2mindFinal['Event']
event2mindFinal['Xemotion_2'] = event2mindFinal['Xemotion']
event2mindFinal['Xintent_2'] = event2mindFinal['Xintent']


In [10]:
cleanDf = event2mindFinal[event2mindFinal['Xintent'] != 'none']
cleanDf = cleanDf[cleanDf['Xintent'] != 'non']


cleanDf.head(15)

,Indice,Xintent,Xemotion,Event,Xsent,Osent,Event_2,Xemotion_2,Xintent_2
Indice2,,,,,,,,,
773,773,to have an advantage,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,pleased,to have an advantage
774,773,to have an advantage,smug,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,smug,to have an advantage
775,773,to have an advantage,excited,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,excited,to have an advantage
776,773,to fulfill a desire,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,pleased,to fulfill a desire
777,773,to fulfill a desire,smug,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,smug,to fulfill a desire
778,773,to fulfill a desire,excited,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,excited,to fulfill a desire
779,773,to get out of trouble,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,pleased,to get out of trouble
780,773,to get out of trouble,smug,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,smug,to get out of trouble
781,773,to get out of trouble,excited,PersonX uses PersonX's ___ to obtain,3.0,2.0,PersonX uses PersonX's ___ to obtain,excited,to get out of trouble


In [11]:
len(cleanDf)

92596

## Aplicando retirada de stop words

In [12]:
  import nltk
  import re
  from nltk.tokenize import word_tokenize
  nltk.download('stopwords')
  nltk.download('rslp')
  nltk.download('punkt')

  def cleanRegExp(entrada):
    for x in range(0,len(entrada)):
      return(re.sub('[\[\]]','', str(entrada)))
 
  stop_words = nltk.corpus.stopwords.words('english')
  stemmer = nltk.stem.RSLPStemmer()

#Trabalhando com stemmer e stopwords
  for x in range(1, len(event2mindFinal['Event_2'])):
#Remover as stopwords do texto
    word_tokens = word_tokenize(str(event2mindFinal['Event_2'][x]))
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
#Remover sufixos
    line = []
    text_tokenized = word_tokenize(str(cleanRegExp(filtered_sentence)))
    line = [stemmer.stem(word) for word in text_tokenized]
    event2mindFinal['Event_2'][x] = cleanRegExp(line)

removeCaracteres('Event_2', event2mindFinal)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Lemmatization para campo de emoções

In [13]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

#Trabalhando com lemmatizer
for x in range(1, len(event2mindFinal['Xemotion_2'])):
#Remover as stopwords do texto
    word_tokens = word_tokenize(str(event2mindFinal['Xemotion_2'][x]))
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
#Remover sufixos

    word_list = nltk.word_tokenize(str(cleanRegExp(filtered_sentence)))
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    #line = []
    #line = [stemmer.stem(word) for word in text_tokenized]
    event2mindFinal['Xemotion_2'][x] = lemmatized_output

removeCaracteres('Xemotion_2', event2mindFinal)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Lematization para campo de intenções

In [14]:
lemmatizer = WordNetLemmatizer()

#Trabalhando com lemmatizer
for x in range(1, len(event2mindFinal['Xintent_2'])):
#Remover as stopwords do texto
    word_tokens = word_tokenize(str(event2mindFinal['Xintent_2'][x]))
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
#Remover sufixos

    word_list = nltk.word_tokenize(str(cleanRegExp(filtered_sentence)))
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    event2mindFinal['Xintent_2'][x] = lemmatized_output

removeCaracteres('Xintent_2', event2mindFinal)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Conferindo a segunda versão (em 12/06) trazendo Osent

In [15]:
cleanDf = event2mindFinal[event2mindFinal['Xemotion'] != "none"]

cleanDf.tail(30)

,Indice,Xintent,Xemotion,Event,Xsent,Osent,Event_2,Xemotion_2,Xintent_2
Indice2,,,,,,,,,
108936,57071,to tell something,satisfied,PersonX calls PersonY's brother,3.0,3.0,"'personx, ', ',', 'call, ', ',', 'persony, ', ...",'satisfied ',"'tell ' , 'something '"
108937,57072,to socialize,satisfied,PersonX calls PersonY's brother,3.0,4.0,"'personx, ', ',', 'call, ', ',', 'persony, ', ...",'satisfied ','socialize '
108938,57073,to talk to him,connected,PersonX calls PersonY's brother,3.0,5.0,"'personx, ', ',', 'call, ', ',', 'persony, ', ...",'connected ','talk '
108939,57074,comfort,comforted,PersonX sucks PersonX's thumb,4.0,5.0,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'comforted ','comfort '
108940,57074,familiarity,comforted,PersonX sucks PersonX's thumb,4.0,5.0,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'comforted ','familiarity '
108941,57075,none,embarrassed,PersonX sucks PersonX's thumb,NaN,1.0,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'embarrassed ','none '
108942,57076,to comfort himself,sleepy,PersonX sucks PersonX's thumb,3.0,3.0,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'sleepy ','comfort '
108943,57077,them to have it for school,happy to help a friend,PersonX leaves PersonY's book,5.0,5.0,"'personx, ', ',', 'leav, ', ',', 'persony, ', ...","'happy ' , 'help ' , 'friend '",'school '
108944,57078,persony to get it,relieved,PersonX leaves PersonY's book,3.0,3.0,"'personx, ', ',', 'leav, ', ',', 'persony, ', ...",'relieved ',"'persony ' , 'get '"


Conferindo base final - não rodar codigo abaixo para fins de comparação - gerado em Maio

In [0]:
cleanDf = event2mindFinal[event2mindFinal['Xemotion'] != "none"]

cleanDf.tail(30)

,Indice,Xintent,Xemotion,Event,Xsent,Event_2,Xemotion_2,Xintent_2
Indice2,,,,,,,,
108936,57071,to tell something,satisfied,PersonX calls PersonY's brother,3.0,"'personx, ', ',', 'call, ', ',', 'persony, ', ...",'satisfied ',"'tell ' , 'something '"
108937,57072,to socialize,satisfied,PersonX calls PersonY's brother,3.0,"'personx, ', ',', 'call, ', ',', 'persony, ', ...",'satisfied ','socialize '
108938,57073,to talk to him,connected,PersonX calls PersonY's brother,3.0,"'personx, ', ',', 'call, ', ',', 'persony, ', ...",'connected ','talk '
108939,57074,comfort,comforted,PersonX sucks PersonX's thumb,4.0,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'comforted ','comfort '
108940,57074,familiarity,comforted,PersonX sucks PersonX's thumb,4.0,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'comforted ','familiarity '
108941,57075,none,embarrassed,PersonX sucks PersonX's thumb,NaN,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'embarrassed ','none '
108942,57076,to comfort himself,sleepy,PersonX sucks PersonX's thumb,3.0,"'personx, ', ',', 'suck, ', ',', 'personx, ', ...",'sleepy ','comfort '
108943,57077,them to have it for school,happy to help a friend,PersonX leaves PersonY's book,5.0,"'personx, ', ',', 'leav, ', ',', 'persony, ', ...","'happy ' , 'help ' , 'friend '",'school '
108944,57078,persony to get it,relieved,PersonX leaves PersonY's book,3.0,"'personx, ', ',', 'leav, ', ',', 'persony, ', ...",'relieved ',"'persony ' , 'get '"


In [0]:
export_path = workdir_path + '/event2MindClean.csv'

event2mindFinal.to_csv (export_path, index = False, header=True)